### Task:
You are tasked with creating a report for failed login attempts on a secure system. Given the table `login_attempts`, which has the following columns:

- `user_id`: ID of the user
- `timestamp`: Date and time of the login attempt
- `status`: Status of the login attempt (either 'success' or 'failure')

Write a SQL query to find all users who had **more than 3 failed login attempts** within a **24-hour period**. For each user, show the `user_id` and the **total count** of failed attempts within that period.

### Schema:
`login_attempts(user_id, timestamp, status)`

---

### SQL Query:

```sql
WITH failed_logins AS (
    SELECT
        user_id,
        timestamp,
        COUNT(*) OVER (PARTITION BY user_id ORDER BY timestamp RANGE BETWEEN INTERVAL '1' DAY PRECEDING AND CURRENT ROW) AS fail_count
    FROM login_attempts
    WHERE status = 'failure'
)
SELECT user_id, COUNT(*) AS total_failed_attempts
FROM failed_logins
WHERE fail_count > 3
GROUP BY user_id;


### Explanation:
1. **`WITH failed_logins`**: This Common Table Expression (CTE) creates a subquery to calculate the number of failed login attempts by each user in a rolling 24-hour window. The `COUNT(*) OVER` function is used to count the failures per user, partitioned by `user_id`, and ordered by the `timestamp`. The `RANGE BETWEEN INTERVAL '1' DAY PRECEDING AND CURRENT ROW` clause defines the 24-hour rolling window.
   
2. **`WHERE status = 'failure'`**: This filters the `login_attempts` table to include only the failed login attempts, ignoring the successful ones.

3. **`RANGE BETWEEN INTERVAL '1' DAY PRECEDING AND CURRENT ROW`**: This part defines the 24-hour period to look at each login attempt in the past 24 hours for a specific user.

4. **`SELECT user_id, COUNT(*) AS total_failed_attempts`**: In the outer query, the `WHERE fail_count > 3` clause filters the results to include only users who had more than 3 failed login attempts. It then groups the results by `user_id` to return the total count of failed login attempts for each user.
